In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Json
import json

In [4]:
vvProdutos = 'https://www.vv.com.br/pt/produto/detalhe/'
codigosVV= []
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''

In [5]:
query = pd.read_csv('produtos_bot_yury_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)
display(df_query)

,ET0100_PRODUTO,ET0100_NOME,ET0100_D_S_I,ET0090_EAN
0,101297033,DEDO CARACOL 334.972 - VV-0183-01,18001001,7895985002025
1,101280608,ENGRENAGEM 1175.165.001.00- VV-0735-00,18001002,7895823110141
2,100415797,ENGRENAGEM H-159.615 - VV-3462-00,18001004,8309266
3,101297491,JUNCAO BARRA CORTE SECCIONADA VV-6200-99,18001004,7895985027981
4,101281421,CUBO 320.053 - 373.929 - VV-0393-00,18001001,7895823111759
...,...,...,...,...
95,100091151,BARRA CORTE 15 PES 826.807-VV-0613-01,18001005,7895985005552
96,100094138,MANCAL 387.449 VV-0485-00,18001001,7895985005064
97,101279664,ENGRENAGEM 423.662 - VV-0460-00,18001001,7895823107721
98,101297125,KIT LAMINA PICADOR 849.91906 VV-0352-00,18001006,7895985003718


In [46]:
def buscaInfo(nav, codigo, index):
    if '-' not in codigo or '/' in codigo:
        return
    keys = []
    values = []
    nav.get(str(vvProdutos + codigo))
    nav.implicitly_wait(0.8)
    
    element = nav.find_elements(By.CLASS_NAME, 'itensEquipamentos')
    
    if not element:
        notFound.loc[len(notFound)] = [index, 'não encontrado no site']
        return
    
    for elemento in element:
        ps = elemento.find_elements(By.TAG_NAME, 'p')
        p = []
        for elemento in ps:
            p.append(elemento.text)
        equips = { index : { p[0] : { 'Código OEM' : p[1], 'Equipamentos': p[2:] } } }
    
    keys.append('ID:')
    values.append(index)
    
    element = nav.find_element(By.CLASS_NAME, 'subtitulo')
    nome = element.find_element(By.TAG_NAME, 'h2').text
    nome = nome.replace('–', '-')
    
    keys.append('Nome:')
    values.append(nome)
    
    element = nav.find_element(By.CLASS_NAME, 'descricao')
    li = element.find_elements(By.TAG_NAME, 'li')    
    
    for elemento in li:
        for k in elemento.find_elements(By.TAG_NAME, 'h3'):
            keys.append(k.text)
        for v in elemento.find_elements(By.TAG_NAME, 'p'):
            texto = v.text
#             texto = texto.replace('"', 'pol')
#             texto = texto.replace("'", 'pol')
            texto = texto.replace('"', '\"')
            texto = texto.replace("'", '\"')
            values.append(texto)
    dict_infos = dict(zip(keys, values))
    element = nav.find_element(By.CLASS_NAME, 'listaGaleria')
    imgs = element.find_elements(By.TAG_NAME, 'img')
    src = []
    for img in imgs:
        src.append(img.get_attribute('src'))
    images = {index : src}
    
    dict_infos = str(dict_infos)
    dict_infos = dict_infos.replace("'", '"')
    images = str(images)
    images = images.replace("'", '"')
    equips = str(equips)
    equips = equips.replace("'", '"')
    with open('infos.json', 'a') as teste:
        teste.write(dict_infos)
        teste.write(',')
    with open('images.json', 'a') as teste:
        teste.write(images)
        teste.write(',')
    with open('equips.json', 'a') as teste:
        teste.write(equips)
        teste.write(',')

In [7]:
def findVVCod(nome):
    indice = nome.find('VV')
    nome = nome[indice:]
    nome = nome.replace(' ', '')
    nome = nome.split('/')
    cods = []
    for cod in nome:
        cod = cod.replace('VV-', '')
        cod = re.sub('[A-Z]', '', cod)
        cods.append(cod)
    return cods

In [39]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)

In [47]:
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findVVCod(cod)
    for item in cods:
        if len(item) > 6:
            try:
                buscaInfo(nav, item, ID)
            except Exception as e:
                print(e)
        else:
            notFound.loc[len(notFound)] = [ID, 'Código sem sufixo']

In [34]:
buscaInfo(nav, '0069-00', '101297095')